<a href="https://colab.research.google.com/github/meahmadkarim/Topical_Chatbot/blob/main/Topical_Chatbot_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pandas nltk

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import string, os
import re
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/NLP Project/Data/topical_chat.csv')
df.head()

,conversation_id,message,sentiment
0,1,Are you a fan of Google or Microsoft?,Curious to dive deeper
1,1,Both are excellent technology they are helpfu...,Curious to dive deeper
2,1,"I'm not a huge fan of Google, but I use it a...",Curious to dive deeper
3,1,Google provides online related services and p...,Curious to dive deeper
4,1,"Yeah, their services are good. I'm just not a...",Curious to dive deeper


In [ ]:
import string

def custom_preprocess(text):
    # Convert to lowercase
    text = text.lower()

    # Replace specific characters with spaces
    for char in ['\n', '-', ':', ',', '"', '.', '!', '?', ';']:
        text = text.replace(char, ' ')

    # Remove punctuation characters
    text = ''.join(char for char in text if char not in string.punctuation)

    # Remove extra spaces
    text = ' '.join(text.split())

    return text


In [ ]:
df.message = df.message.apply(custom_preprocess)

In [ ]:
# Vectorize the data.
X = []
Y = []
input_words_set = set()
target_words_set = set()

for conversation_index in tqdm(range(df.shape[0])):

    if conversation_index == 0:
        continue

    x = df.iloc[conversation_index - 1]
    y = df.iloc[conversation_index]

    if x.conversation_id == y.conversation_id:

        x = x.message
        y = y.message

        if len(x.split()) > 2 and \
            len(y.split()) > 0 and \
            len(x.split()) < 30 and \
            len(y.split()) < 10 and \
            x and \
            y:

            y = "str  " + y + " end"

            X.append(x)
            Y.append(y)

            for word in x.split():
                if word not in input_words_set:
                    input_words_set.add(word)
            for word in y.split():
                if word not in target_words_set:
                    target_words_set.add(word)

100%|██████████| 188378/188378 [00:42<00:00, 4468.91it/s]


In [ ]:
import tensorflow_datasets as tfds

import os
import re
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
MAX_SENTENCE_LENGTH = 60

In [ ]:
import tensorflow_datasets as tfds

# Your data preparation code here
# ...

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    X + Y, target_vocab_size=2**13)

START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]


In [ ]:
def tokenize_and_filter(X, Y):
  tok_input, tok_output = [], []

  for (sen, sen1) in zip(X, Y):
    # tokenize sentence
    sen = START_TOKEN + tokenizer.encode(sen) + END_TOKEN
    sen1 = START_TOKEN + tokenizer.encode(sen1) + END_TOKEN
    # check tokenized sentence max length
    if len(sen) <= MAX_SENTENCE_LENGTH and len(sen1) <= MAX_SENTENCE_LENGTH:
      tok_input.append(sen)
      tok_output.append(sen1)

  # pad tokenized sentences
  tok_input = tf.keras.preprocessing.sequence.pad_sequences(
      tok_input, maxlen=MAX_SENTENCE_LENGTH, padding='post')
  tok_output = tf.keras.preprocessing.sequence.pad_sequences(
      tok_output, maxlen=MAX_SENTENCE_LENGTH, padding='post')

  return tok_input, tok_output


X, Y = tokenize_and_filter(X, Y)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': X,
        'dec_inputs': Y[:, :-1]
    },
    {
        'outputs': Y[:, 1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
def scaled_attention(query, key, value, mask):
  """Calculate the attention weights. """
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # scale matmul_qk
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask to zero out padding tokens
  if mask is not None:
    logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k)
  weight = tf.nn.softmax(logits, axis=-1)

  output = tf.matmul(weight, value)

  return output

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # scaled dot-product attention
    scaled_attention = scaled_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # concatenation of heads
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # final linear layer
    outputs = self.dense(concat_attention)

    return outputs

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:

class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
sample_encoder_layer = encoder_layer(
    units=512,
    d_model=256,
    num_heads=8,
    dropout=0.1,
    name="our_encoder_layer")

tf.keras.utils.plot_model(
    sample_encoder_layer, to_file='encoder_layer.png', show_shapes=True)

UnboundLocalError: ignored